Gives access to Google Drive env

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Installing Packages

In [3]:
# Install the Ultralytics package (includes YOLOv8 framework)
# This library provides tools for object detection, classification, and segmentation.
# It will be used to train and evaluate our deep learning model efficiently.
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 30.0 MB/s eta 0:00:00


Configuring and Unzipping the YOLO Training Dataset

In [4]:
# --- CONFIGURATION ---
ZIP_FILE_NAME = "My Yolo Detection DB.zip"
# ---------------------

# Copy the zip file from Drive
!cp "/content/drive/MyDrive/{ZIP_FILE_NAME}" "/content/"

# Unzip
!unzip -q "/content/{ZIP_FILE_NAME}" -d "/content/"

print(f" Successfully unzipped {ZIP_FILE_NAME} into /content/")
print("Your unzipped files are:")
!ls /content

 Successfully unzipped My Yolo Detection DB.zip into /content/
Your unzipped files are:
 drive	'My Yolo Detection DB'	'My Yolo Detection DB.zip'   sample_data


YAML Configuration Update for YOLO Dataset Paths

In [5]:
import yaml

# --- CONFIGURATION ---
PROJECT_FOLDER = "My Yolo Detection DB"
YAML_FILE_NAME = "v1_dataset.yaml"
DATASET_FOLDER_NAME = "_03_split_dataset"


yaml_file_path = f"/content/{PROJECT_FOLDER}/{YAML_FILE_NAME}"
new_data_path = f"/content/{PROJECT_FOLDER}/{DATASET_FOLDER_NAME}"

try:
    with open(yaml_file_path, 'r') as f:
        data = yaml.safe_load(f)

    data['path'] = new_data_path

    data['train'] = "train/images"
    data['val'] = "valid/images"
    data['test'] = "test/images"


    with open(yaml_file_path, 'w') as f:
        yaml.dump(data, f)

    print(f"Successfully updated {yaml_file_path}")
    print("--- New Content: ---")
    !cat "{yaml_file_path}"

except Exception as e:
    print(f"ERROR: Could not find or update YAML file.")
    print(f"Searched for: {yaml_file_path}")

Successfully updated /content/My Yolo Detection DB/v1_dataset.yaml
--- New Content: ---
names:
  0: bottle
  1: can
  2: detergent
  3: pulses
  4: seafood
  5: fruit
nc: 6
path: /content/My Yolo Detection DB/_03_split_dataset
test: test/images
train: train/images
val: valid/images


YOLOv8 Model Training Configuration and Execution

In [ ]:
# Train YOLOv8n model
!yolo task=detect mode=train \
    model=yolov8n.pt \
    data="/content/My Yolo Detection DB/v1_dataset.yaml" \
    epochs=75 \
    imgsz=640 \
    batch=-1 \
    project="/content/training_runs" \
    name="fresh_train_v1" \
    degrees=15.0 \
    hsv_s=0.5 \
    hsv_v=0.3 \
    fliplr=0.5 \
    flipud=0.0 \
    mosaic=1.0 \
    mixup=0.1

# ---------------------------------------------------
# Save YOLO Training Results (.pt, metrics, etc.) to Google Drive
# ---------------------------------------------------

# Create a folder in your Drive to store all results
!mkdir -p "/content/drive/MyDrive/YOLO_Fresh_Training_Results"

# Copy the entire training folder (including weights/best.pt) to Drive
!cp -r "/content/training_runs/fresh_train_v1" "/content/drive/MyDrive/YOLO_Fresh_Training_Results/"

print("✅ Training complete!")
print("📁 Your model weights and results (including 'best.pt') are now saved in:")
print("➡ /content/drive/MyDrive/YOLO_Fresh_Training_Results/fresh_train_v1/weights/")


Ultralytics 8.3.227 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/My Yolo Detection DB/v1_dataset.yaml, degrees=15.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=75, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.5, hsv_v=0.3, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=fresh_train_v12, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspec

Model Evaluation

In [7]:
# ==========================================================
# ========== YOLOv8 DETECTION — MODEL EVALUATION ===========
# ==========================================================
# Evaluates a YOLOv8 *detection* model (.pt) on the test split
# defined in your data YAML.
# Automatically copies evaluation results to Google Drive.
# ==========================================================

import os
import shutil
import torch
from pathlib import Path
import numpy as np
from ultralytics import YOLO

# ---------------- Config ----------------
MODEL_PATH = "/content/drive/MyDrive/YOLO_Fresh_Training_Results/fresh_train_v1/weights/best.pt"
DATA_YAML  = "/content/My Yolo Detection DB/v1_dataset.yaml"   # your dataset yaml
SPLIT      = "test"   # or 'val'
IMGSZ      = 640
BATCH      = 16
IOU_THRES  = 0.6
CONF_THRES = 0.001
DEVICE     = 0 if torch.cuda.is_available() else "cpu"

# Destination folder in Google Drive
DRIVE_SAVE_DIR = "/content/drive/MyDrive/YOLO_Evaluation_Results"

# ------------- Evaluate -----------------
def evaluate_yolov8_detection():
    print("\n--- Starting YOLOv8 DETECTION Evaluation ---\n")

    model = YOLO(MODEL_PATH)
    names = model.names  # {idx: class_name}
    print(f"Loaded model with {len(names)} classes.\n")

    metrics = model.val(
        data=DATA_YAML,
        split=SPLIT,
        imgsz=IMGSZ,
        batch=BATCH,
        iou=IOU_THRES,
        conf=CONF_THRES,
        device=DEVICE,
        plots=True,
        save_hybrid=True,
        save_json=True,
        verbose=False
    )

    # Local folder where YOLO saves results
    save_dir = Path(getattr(metrics, "save_dir", "runs/detect/val"))
    print(f"\n📁 Evaluation results saved locally to: {save_dir}")

    # -------- Overall metrics --------
    box = metrics.box
    overall = {
        "Precision": getattr(box, "mp", None),
        "Recall":    getattr(box, "mr", None),
        "mAP@0.50":  getattr(box, "map50", None),
        "mAP@0.50:0.95": getattr(box, "map", None),
    }

    print("\n=== OVERALL PERFORMANCE ===")
    for k, v in overall.items():
        print(f"{k:<15}: {float(v):.4f}" if v is not None else f"{k:<15}: N/A")

    # -------- Per-class AP --------
    maps = np.array(getattr(box, "maps", []))
    print("\n=== PER-CLASS AP (mAP@0.50:0.95) ===")
    print(f"{'Class':<25}{'AP':>8}")
    print("-" * 36)
    for i, ap in enumerate(maps):
        cname = names.get(i, f"class_{i}")
        print(f"{cname:<25}{ap:>8.3f}")
    print("-" * 36)

    # -------- Copy results to Drive --------
    drive_path = Path(DRIVE_SAVE_DIR) / save_dir.name
    os.makedirs(drive_path, exist_ok=True)
    shutil.copytree(save_dir, drive_path, dirs_exist_ok=True)

    print(f"\n✅ All evaluation results have been copied to your Google Drive:")
    print(f"➡ {drive_path}\n")

    print("Files include:")
    print("• confusion_matrix.png")
    print("• PR_curve.png, F1_curve.png, P_curve.png, R_curve.png")
    print("• results.csv (raw metrics)")
    print("• predictions.json (COCO metrics if save_json=True)")
    print("\n📊 Evaluation complete and safely backed up to Drive!")

if __name__ == "__main__":
    evaluate_yolov8_detection()



--- Starting YOLOv8 DETECTION Evaluation ---

Loaded model with 6 classes.

WARNING ⚠️ 'save_hybrid' is deprecated and will be removed in the future.
Ultralytics 8.3.227 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,818 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1027.1±321.2 MB/s, size: 24.3 KB)
val: Scanning /content/My Yolo Detection DB/_03_split_dataset/test/labels.cache... 1515 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1515/1515 2.6Mit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 95/95 6.6it/s 14.4s
                   all       1515       1652      0.923      0.856      0.931      0.725
Speed: 1.1ms preprocess, 3.6ms inference, 0.0ms loss, 1.1ms postprocess per image
Saving /content/runs/detect/val2/predictions.json...
Results saved to /content/runs/detect/val2

📁 Evaluation results saved locally to: /co